# **1. Setup : Install required packages datasets, evaluate and transformers[sentencepiece].**



In [1]:
%pip install --quiet datasets evaluate transformers[sentencepiece]

# **2. Load & Inspect Dataset :**

In [2]:
!pip install --quiet -U datasets

In [3]:
from datasets import load_dataset
import pandas as pd

raw = load_dataset("sms_spam")


train_ds = raw['train'].shuffle(seed=42).select(range(4000))
val_ds   = raw['train'].shuffle(seed=42).select(range(4000, 5000))

train_ds.features

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'sms': Value('string'), 'label': ClassLabel(names=['ham', 'spam'])}

# **3. Tokenization :**

In [4]:
from transformers import GPT2Tokenizer

In [5]:
model_name = "gpt2"
tokenizer  = GPT2Tokenizer.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token

def tokenize_fn(examples):

    return tokenizer(
        examples["sms"],
        padding="max_length",
        truncation=True,
        max_length=64
    )

train_tok = train_ds.map(tokenize_fn, batched=True)
val_tok   = val_ds.map(tokenize_fn, batched=True)

# **4. Model Initialization**

In [6]:
import torch
from transformers import GPT2ForSequenceClassification

model = GPT2ForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    pad_token_id=tokenizer.eos_token_id
)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# **5. Metrics Definition**

In [7]:
import evaluate
import numpy as np

accuracy  = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall    = evaluate.load("recall")
f1        = evaluate.load("f1")

def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=-1)
    return {
        "accuracy":  accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "precision": precision.compute(predictions=preds, references=labels, average = "binary")["precision"],
        "recall":    recall.compute(predictions=preds, references=labels, average = "binary")["recall"],
        "f1":        f1.compute(predictions=preds, references=labels, average = "binary")["f1"]
    }

Dans un jeu de données déséquilibré comme SMS spam (avec beaucoup plus de "ham" que de "spam"), l'accuracy seule peut être trompeuse : un modèle qui prédit toujours "ham" aura une bonne accuracy sans détecter les spams. Suivre aussi la précision et le recall est essentiel. Un modèle avec une haute accuracy mais un faible rappel sur le spam signifie qu’il rate beaucoup de spams : il est trop prudent et en laisse passer, ce qui est problématique pour la détection.

# **6. TrainingArguments Configuration**

In [8]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/results",
    do_train=True,
    do_eval=True,
    eval_steps=500,
    save_steps=500,
    logging_dir="/content/logs",
    logging_steps=500,

    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,

    report_to=None,
    save_total_limit=1,
)

Le weight_decay limite la taille des poids du modèle pour éviter l’overfitting. Une valeur plus élevée est utile si le modèle apprend trop bien les données d’entraînement mais généralise mal. Une valeur plus basse est utile si le dataset est petit ou si le modèle a du mal à apprendre.

In [9]:
!pip uninstall -y wandb

# **7. Train & Evaluate**

In [10]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    compute_metrics=compute_metrics,
)
trainer.train()

metrics = trainer.evaluate()
print(metrics)

Step,Training Loss
500,0.115000
1000,0.044500
1500,0.026700


{'eval_loss': 0.03444773703813553, 'eval_accuracy': 0.993, 'eval_precision': 0.991304347826087, 'eval_recall': 0.95, 'eval_f1': 0.9702127659574468, 'eval_runtime': 3.9394, 'eval_samples_per_second': 253.845, 'eval_steps_per_second': 31.731, 'epoch': 3.0}


Le modèle est très performant. Il prédit correctement dans 99,3 % des cas. Il identifie presque tous les spams (rappel 95 %) et se trompe très peu quand il dit qu’un message est un spam (précision 99,1 %). Le F1-score de 97 % montre un bon équilibre entre les deux.